In [1]:
# Dependencies
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import pymongo
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

# First part of the homework #
## find the top news story headline and the summary ##

In [2]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [3]:
# Define database and collection
db = client.mars_db
collection = db.articles

In [4]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/'

# Retrieve page with the requests module
response = requests.get(url)
# Create BeautifulSoup object; parse with 'html.parser'
soup = bs(response.text, 'lxml')

In [ ]:
# Retrieve the parent divs for all articles
results = soup.find_all('div', class_='slide')

# loop through results to get items
for result in results:

    title = result.find('div', class_='content_title').text

    paragraph = result.find('div', class_='rollover_description_inner').text

    date = result.find('div', class_='list_date')


    print('-----------------')
    print(title)
    print(paragraph)
    print(date) #date isn't found.  on webpage there is a div but not in response

In [5]:
# assign response to variables and print

news_title = soup.find('div', class_='content_title').text
news_p = soup.find('div', class_='rollover_description_inner').text

print('----------')
print(news_title)
print(news_p)

----------


NASA Ingenuity Mars Helicopter Prepares for First Flight



Now uncocooned from its protective carbon-fiber shield, the helicopter is being readied for its next steps.  



In [ ]:
print(news_title)
print(news_p)

 # Second part of homework (get the featured image url from JPL page)#

In [ ]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

In [ ]:
url = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html'
browser.visit(url)

In [ ]:
# HTML object
html = browser.html

# Parse HTML with Beautiful Soup
soup = bs(html, 'html.parser')
# find the parent div of the image
img_url = soup.find('div', class_='floating_text_area')
# find the href within the div
ref = img_url.find('a')['href']
# build the final link for the featured image
featured_image_url = ['https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/' + ref]

In [ ]:
print(featured_image_url)

In [ ]:
browser.quit()

# Third part of homework (use pandas to get table from webpage)#

In [ ]:
url = 'https://space-facts.com/mars/'

In [ ]:

tables = pd.read_html(url)
table = tables[0]
table

In [ ]:
table.to_html('table.html')

In [ ]:
!open table.html

# Mars Hemispheres #

In [ ]:
# URL of page to be scraped
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

# Retrieve page with the requests module
response = requests.get(url)
# Create BeautifulSoup object; parse with 'html.parser'
soup = bs(response.text, 'html.parser')


In [ ]:
results = soup.find_all('div', class_='item')

In [ ]:
# iterate through results to get title and href of images
# found solution for finding href here:  https://stackoverflow.com/questions/38570411/how-to-scrape-href-with-python-3-5-and-beautifulsoup

base = 'https://astrogeology.usgs.gov/'
for result in results:
    print(result.find('h3').text)
    print(base + (result.a['href']) + '.tif')


In [ ]:
 title_obj = [i.h3.text for i in results]
 href_obj = [base + i.a['href'] + '.tif' for i in results]
#  for i in soup.find_all('div', class_='item'):
#      print(i.a['href'])
print(href_obj)
print(title_obj)

In [ ]:
# hemisphere_image_urls = {}.fromkeys(['title','img_url'],(title_obj, href_obj))
# hemisphere_image_urls
hemisphere_image_urls = []
for i in range(len(title_obj)):
    dict = {}
    dict['title'] = title_obj[i]
    dict['img_url'] = href_obj[i]
    hemisphere_image_urls.append(dict)
print(hemisphere_image_urls)
